<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/6_3_Cargar_y_trabajar_con_modelos_preentrenados_de_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 6.3 Cargar y trabajar con modelos preentrenados de Huggingface

Existen cientos de modelos ya preentrenados (con Finetuning) para distintas tareas en Huggingface (https://huggingface.co/) tanto de modelos BERT/RoBERTa como otros modelos de transformers como GPT.

Estos modelos no son únicamente para tareas de PLN sino que hay otros modelos de procesamiento de imágenes, etc.

En esta sesión vamos a cargar y probar 2 modelos para distintas tareas de PLN como es el análisis de sentimientos y la detección de entidades (NER)

Como siempre, instalamos las librerías de transformers y datasets

In [1]:
# Install libraries
!pip3 install -U transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00


## Apartado 1.1 Descargamos y probamos un modelo de análisis de sentimientos en español

El modelo en cuestión es el que se encuentra en la página https://huggingface.co/finiteautomata/beto-sentiment-analysis y es un finetuning de un modelo BETO usando el dataset del TASS 2020.

In [2]:
import transformers
from transformers import AutoModelForSequenceClassification

path_sentiment_model = 'finiteautomata/beto-sentiment-analysis'

# Obtenemos el tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained (path_sentiment_model)
# Obtenemos el modelo para análisis de sentimientos que tiene 3 etiquetas: positivo, negativo y neutro
sentiment_model = AutoModelForSequenceClassification.from_pretrained(path_sentiment_model)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Apartado 1.2 Probamos el modelo de análisis de sentimientos con un conjunto de textos.

Obtenemos la clasificación del sentimiento a partir de un conjunto de textos.

Estos modelos a los que ya se les ha hecho un finetuning se pueden volver a entrenar con otros finetuning.

In [3]:
import torch

# Probamos a clasificar estas frases
textos = ['la película fue un rollazo y me aburrí mucho',
          'me encanta este actor',
          'la pelicula dura 2 horas',
          'estoy encantado con el regalo que me han hecho',
          'hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# Se obtienen los sentimientos de los textos
for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = sentiment_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= sentiment_model.config.id2label[predicted_class_id]

  print(text,'=>', prediction)


la película fue un rollazo y me aburrí mucho => NEG
me encanta este actor => POS
la pelicula dura 2 horas => NEU
estoy encantado con el regalo que me han hecho => POS
hay muchos más muertos por covid => NEG
el número de afectados por covid aumenta => NEG
vamos a salir de la pandemia => NEU
ánimo a todos => POS


## Apartado 1.3 Probamos ahora un modelo de NER en español

A continuación probamos el modelo de reconocimiento de entidades en español NER (https://huggingface.co/PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus) entrenado con el dataset de CAPITEL

In [4]:
spanish_ner_model = 'PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus'

from transformers import AutoModelForTokenClassification


# Obtenemos el tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained (spanish_ner_model)
# Obtenemos el modelo para la extracción de entidades
ner_model = AutoModelForTokenClassification.from_pretrained(spanish_ner_model)


# Apartado 1.4 Probamos como extraer entidades de distintos textos

Probamos cómo puede extraer las entidades de distintos textos según los tokens obtenidos por el tokenizer de BERT.

In [5]:
from transformers import pipeline

nlp = pipeline("ner", model=ner_model, tokenizer=tokenizer)
examples = ["Soy Rafael Valencia profesor de la asignatura TGINE del máster en Big Data.",
            "Carlos Alcaraz es un tenista de Murcia que está destacando en las competiciones ATP",
            "Mercadona es una cadena de supermercados que está en toda España"
]

ner_results = nlp(examples)
for ner_text, i in zip(ner_results, range(len(ner_results ))):
  print(examples[i])
  for ner in ner_text:
    print(ner)

Soy Rafael Valencia profesor de la asignatura TGINE del máster en Big Data.
{'entity': 'B-PER', 'score': 0.9997787, 'index': 2, 'word': 'ĠRafael', 'start': 4, 'end': 10}
{'entity': 'E-PER', 'score': 0.9997545, 'index': 3, 'word': 'ĠValencia', 'start': 11, 'end': 19}
{'entity': 'S-OTH', 'score': 0.99898344, 'index': 8, 'word': 'ĠT', 'start': 46, 'end': 47}
{'entity': 'S-OTH', 'score': 0.9925033, 'index': 9, 'word': 'G', 'start': 47, 'end': 48}
{'entity': 'E-OTH', 'score': 0.53592414, 'index': 10, 'word': 'INE', 'start': 48, 'end': 51}
{'entity': 'B-OTH', 'score': 0.998509, 'index': 14, 'word': 'ĠBig', 'start': 66, 'end': 69}
{'entity': 'E-OTH', 'score': 0.9967908, 'index': 15, 'word': 'ĠData', 'start': 70, 'end': 74}
Carlos Alcaraz es un tenista de Murcia que está destacando en las competiciones ATP
{'entity': 'B-PER', 'score': 0.9998116, 'index': 1, 'word': 'ĠCarlos', 'start': 0, 'end': 6}
{'entity': 'E-PER', 'score': 0.99980897, 'index': 2, 'word': 'ĠAlcaraz', 'start': 7, 'end': 14}
{

## Apartado 1.5 Modelos para resumen de texto
Probamos un ejemplo de resumidor de texto basado en modelos RoBERTa
https://huggingface.co/Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization

In [6]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

text = """BONO TURÍSTICO DE LA REGIÓN DE MURCIA: ASÍ PUEDES CONSEGUIR EL 50%
DE DESCUENTO

La Consejería de Turismo, Cultura, Juventud y Deportes aprobó
recientemente los nuevos bonos turísticos con descuentos en
alojamientos de la Región de Murcia. La iniciativa, que se activó el
pasado 12 de noviembre y permanecerá vigente hasta el 31 de mayo de
2024, ofrecerá ayudas directas para cubrir el coste de los servicios
subvencionables del viaje.

Esta tercera edición busca, como las anteriores de 2021 y 2022,
promover la llegada de turistas a la Comunidad durante todo el año.
En esta ocasión, es la primera vez que han optado por iniciarla en la
temporada de otoño-invierno. Con este objetivo, se ha puesto en
circulación un total de 2.500 bonos, que se otorgarán por orden de
formalización de reservas.

Este bono ofrecerá a sus beneficiarios un descuento del 50%, con un
máximo de 250 euros (IVA incluido). Para solicitarlo se deben
contratar unos servicios mínimos, que corresponden a una estancia de,
al menos, dos noches consecutivas en el mismo alojamiento adherido al
programa.

QUIÉN PUEDE SOLICITARLO

Estas ayudas van dirigidas a cualquier persona mayor de edad con
residencia legal y continuada en España. También está contemplado
para ciudadanos de la Región de Murcia. Cada viajero podrá solicitar
más de un bono dentro de cada uno de los periodos de vigencia y
deberá haber pasado un total de diez días naturales desde el momento
en el que realizó la última solicitud. Los viajes no podrán tener
carácter profesional.

Estos periodos de vigencia se dividen en dos. El primero, que parte
desde el día posterior a la publicación de la convocatoria en el
BORM hasta el 22 de diciembre de 2023. Por su parte, el segundo está
activo desde el día siguiente a la publicación en el BORM y hasta el
31 de mayo de 2024. Eso sí, la subvención debe canjearse en el mismo
periodo en el que se adquiere.

CÓMO SE SOLICITA

En la página web www.turismoregiondemurcia.es/es/bono_turistico se
encuentra un apartado llamado 'adquiere tu bono'. Al acceder, se
desplegará un formulario de inscripción, en el que se deben rellenar
todos los datos personales de la persona beneficiaria y completar una
encuesta sobre la intención del viaje. Asimismo, en este documento se
deberá aceptar el contenido de la declaración responsable, que se
podrá descargar y rellenar en un enlace que se aporta dentro de este
mismo formulario.

Una vez que se hayan completado todos los campos, se pulsará sobre
'registrarme'. Acto seguido, recibirá un correo electrónico con un
enlace de confirmación. Este paso es fundamental, ya que de no
confirmar ese registro, no se podrá acceder a las ventajas de la
ayuda.

Al validar la inscripción, recibirá otro correo con el código
promocional o el número de la lista de espera (si se da la
circunstancia de que los bonos se han agotado). Cabe recordar que si
no se formaliza una reserva en un plazo de diez días naturales desde
su adquisición, este caducará y deberá volver a solicitarse.
Además, desde la fecha en la que el alojamiento confirme la reserva
hasta el primer día en el que se disfrute el servicio, deberá
existir un mínimo de siete días naturales y un máximo de 20.

QUÉ SERVICIOS SON SUBVENCIONABLES

Las estancias que forman parte del programa se encuentran en la
tipología de establecimientos hoteleros, apartamentos turísticos,
campings, casas rurales, hospederías rurales y albergues. En la web
oficial de turismo de Murcia se encuentran todos los adscritos a esta
iniciativa.

Los servicios subvencionados que están contemplados en esta ayuda son
los siguientes:

- Manutención, como máximo en régimen de desayuno y con exclusión
de bebidas alcohólicas.

· Tratamientos de salud y bienestar del alojamiento.

· Alquiler de instalaciones y/o equipamiento deportivo del
establecimiento.

· Actividades de animación o deportivas prestadas por el personal
del establecimiento.

- Aparcamiento."""

generate_summary(text)

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


'La Consejería de Cultura, Cultura y Juventud de la Comunidad de Murcia ofrece 2.500 bonos turísticos con descuentos del 50%'

## Apartado 1.6 Modelos para otras tareas

Podéis probar otros modelos para otras tareas distintas consultando cómo se implementaría la llamada a los mismos https://huggingface.co/transformers/usage.html